**Before heading to this, get an idea of AsyncIO**

#### **OPENAI AGENTS SDK**

1. Lightweight and flexible 
2. Stays out of the way 
3. Not opinionated, there are no such rules, terminologies, flows that we need to adhere to in order to use it, so more flexibility
4. Makes common activites easy(creating tools, handles all the json things and conditions)


#### **Minimal Terminologies for Openai Agent SDK** 

1. Agents represents LLMs
2. Handoffs represents interaction 
3. Guardrails represents controls

#### **In order to run an Agent using this SDK, there are three steps**

1. Create an Instance of Agent
2. Use "with trace() to track the agent 
3. Call "runner.run()" to run the agent

The documentation on OpenAI Agents SDK is really clear and simple: <a href="https://openai.github.io/openai-agents-python/">https://openai.github.io/openai-agents-python/</a> and it's well worth a look.

In [4]:
# The imports
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput, WebSearchTool
from agents.model_settings import ModelSettings
from IPython.display import display, Markdown
from openai import AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent  #helps in streaming back text
from typing import Dict
import asyncio
import sendgrid
from pydantic import BaseModel
import os
from sendgrid.helpers.mail import Mail, Email, To, Content

load_dotenv(override=True)

True

In [5]:
#Loading all the api keys from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

In [6]:
# Make an agent with name, instructions, model

agent = Agent(name="Jokester", instructions="You are a joke teller", model="gpt-4o-mini")

#instructions is not but just the system prompt, which provides an agent its idenetiy

#we can use model from other providers like anthropic, cohere, etc. Its not limited to openai.

In [ ]:
agent

# Agent(name='Jokester', 
# handoff_description=None, 
# tools=[], 
# mcp_servers=[], 
# mcp_config={}, 
# instructions='You are a joke teller', 
# prompt=None, 
# handoffs=[],
#  model='gpt-4.1-nano', 
# model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), 
# input_guardrails=[], 
# output_guardrails=[], 
# output_type=None,
# hooks=None, 
# tool_use_behavior='run_llm_again', 
# reset_tool_choice=True)


#this are all the things that can be configured for an agent.

Agent(name='Jokester', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a joke teller', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [5]:
result = Runner.run(agent, input="Tell me a joke about computers.")
print(result)

#returns a coroutine object, as Runner.run is async function. We can use await to get the result.

<coroutine object Runner.run at 0x715013977240>


In [ ]:
result = await Runner.run(agent, input="Tell me a joke about computers.")
print(result.final_output)

Why did the computer break up with the internet?

Because it found someone with better connections!


/tmp/ipykernel_223319/1829248491.py:1: RuntimeWarning: coroutine 'Runner.run' was never awaited
  result = await Runner.run(agent, input="Tell me a joke about computers.")


[non-fatal] Tracing: request failed: _ssl.c:983: The handshake operation timed out
[non-fatal] Tracing: request failed: [Errno -3] Temporary failure in name resolution
[non-fatal] Tracing: request failed: [Errno -3] Temporary failure in name resolution
[non-fatal] Tracing: max retries reached, giving up on this batch.
[non-fatal] Tracing: request failed: [Errno -3] Temporary failure in name resolution
[non-fatal] Tracing: request failed: [Errno -3] Temporary failure in name resolution
[non-fatal] Tracing: request failed: [Errno -3] Temporary failure in name resolution
[non-fatal] Tracing: max retries reached, giving up on this batch.


In [7]:
with trace("Telling a joke"):
    result = await Runner.run(agent, "Tell a joke about Autonomous AI Agents")
    print(result.final_output)

Why did the autonomous AI agent break up with its human partner?

Because it found someone with better "data" compatibility!


#### Now go and look at the trace

https://platform.openai.com/traces

**Leys build a simple Agent system for generating cold sales outreach emails:**
1. Agent workflow
2. Use of tools to call functions
3. Agent as tools
4. Agent collaboration via Tools and Handoffs

**Before we start - some setup:**

Please visit Sendgrid at: https://sendgrid.com/ <br>
Get the api key


In [3]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    print()
    from_email = Email("sonudevmail16@gmail.com") 
    to_email = To("abhinavsarkar53@gmail.com")  
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()


202


**Step-1: Agent Workflow**

In [9]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [10]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [11]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

#this is nothing just a way to use the streaming option for the model, usimg Runner.run_streamed 
#then using async for event 

Subject: Simplify Your SOC 2 Compliance Journey

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am with ComplAI, a leading provider of AI-powered solutions tailored for organizations like yours that strive for SOC 2 compliance.

Navigating the complexities of SOC 2 requirements can be daunting, especially with the need for accurate documentation and timely audits. Our platform streamlines this process, helping you ensure compliance efficiently while reducing the risk of oversights that could lead to costly setbacks.

With ComplAI, you can:

- Automate documentation processes to save time and resources
- Access real-time compliance insights to stay ahead of audits
- Improve collaboration across your teams to enhance accountability

I would love to schedule a brief call to discuss how ComplAI can specifically address your compliance challenges. Are you available for a quick conversation next week?

Thank you for considering this opportunity to

In [12]:
#using all three agents in parallel to write cold sales emails, using asyncio.gather

message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Streamline Your SOC2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent ComplAI, where we specialize in simplifying the SOC2 compliance process through our AI-powered SaaS solution.

As you’re likely aware, maintaining SOC2 compliance can be both time-consuming and complex, often diverting resources from your core business objectives. Our platform is designed to automate key aspects of the compliance journey, ensuring you’re audit-ready without the usual headaches.

Here are a few ways ComplAI can benefit your organization:

- **Automated Documentation**: Reduce manual work with smart document generation based on your specific business practices.
- **Real-Time Monitoring**: Stay on top of compliance standards continuously, rather than waiting for audit time.
- **Guided Audit Preparation**: Streamline your audits with tailored guidance, making the process quicker and more efficient.

I would love the oppo

In [13]:
#creating another agent to pick the best email from the three generated above 

sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [14]:
#here we first run all three sales agents in parallel to generate three different cold sales emails using asyncio.gather 
#then we pass all three emails to the sales_picker agent to pick the best one. using single await Runner.run

message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")

#this was just python code maintaining the flow
#since we have put all these in a single trace we can see it in the trace at openai 


Best sales email:
Subject: Let’s Make SOC2 Compliance a Walk in the Park 🏞️

Hi [Recipient's Name],

I hope this email finds you sipping coffee and enjoying a delightful day! ☕️

I know compliance can feel a bit like trying to assemble IKEA furniture—lots of confusing parts, questionable instructions, and maybe some questionable language along the way. But what if I told you there’s an easier way to get your SOC2 compliance in tip-top shape?

Meet ComplAI, the AI-powered superhero of compliance. 🦸‍♂️ With our SaaS tool, you can say goodbye to late-night auditing nightmares and hello to streamlined processes that practically run themselves. Think of us as your compliance sidekick—only without the spandex.

Here’s how we can help you:

1. **Automated Processes**: No more manual labor! Let our AI do the heavy lifting while you focus on the fun stuff—like plotting world domination (or, you know, your real business goals).
  
2. **Real-time Dashboards**: Keep track of your compliance status

**Part 2: Use of tools**

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [15]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

**Steps 2 and 3: Tools and Agent interactions**

Remember all that boilerplate json?<br>
Simply wrap your function with the decorator `@function_tool`

In [ ]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("sonudevmail16@gmail.com") 
    to_email = To("abhinavsarkar53@gmail.com")   
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

##### **This has automatically been converted into a tool, with the boilerplate json created**

In [17]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x715013738680>, strict_json_schema=True, is_enabled=True)

#### **And you can also convert an Agent into a tool**

In [18]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x715011fb45e0>, strict_json_schema=True, is_enabled=True)

#### **So now we can gather all the tools together:**

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [19]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7150129bc5e0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x71501210ac00>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'req

#### And now it's time for our Sales Manager - our planning agent

In [20]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

### **Handoffs**

#### Handoffs vs. Agents-as-tools

Handoffs are a way for one agent to **delegate responsibility** to another agent, transferring control completely.

Agents-as-tools and handoffs are similar because both involve collaboration between agents:

* **Agents-as-tools** → An agent temporarily calls another agent to perform a task, but control returns to the original agent once the task is done.
* **Handoffs** → An agent fully transfers control to another agent, and the second agent continues the interaction without returning.

**Example:**

* Suppose you’re chatting with a **general customer support agent**.
* If the support agent uses an **internal billing agent as a tool**, it queries that agent for your invoice details, then returns to you with the answer.
* If the support agent performs a **handoff to the billing agent**, the billing agent takes over the conversation directly and continues helping you with payment issues, without the first agent returning.

In [21]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [22]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("sonudevmail16@gmail.com") 
    to_email = To("abhinavsarkar53@gmail.com") 
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [23]:
tools = [subject_tool, html_tool, send_html_email]
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x71501210ab60>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x71501210bd80>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sale

In [24]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")  #this is similiar to the tool description, with this it announce to other agents it capabilities.

#### **Now we have 3 Agents as tools and 1 handoff Agent**

In [25]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7150129bc5e0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x71501210ac00>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'requi

In [26]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

**Now lets look at some more important things**

1. Using models other than openai with openai agent kit 
2. Structured outputs 
3. Guardrails

In [27]:
#the same instructions as before 

instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

#### **It's easy to use any models with OpenAI compatible endpoints**

In [28]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [29]:
#now we will have to create the client for all the different model providers 

openrouter_client = AsyncOpenAI(api_key=openrouter_api_key, base_url="https://openrouter.ai/api/v1")  
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

In [30]:
#now we will have to create models which will compatible with openai agents sdk using openaichatcompletion class
deepseek_model = OpenAIChatCompletionsModel(model="deepseek/deepseek-r1-0528:free", openai_client=openrouter_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [31]:
#now instead of gpt-4o-mini in all the agents, we will use each of these models for different agents. 

sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model) 
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

#here in the param model=, if we pass a string here like "gpt-4o-mini" it will assume that we are using openai models, and use openai client by default.
# But if we pass an object of OpenAIChatCompletionsModel class, it will use that model and its client.

In [32]:
#lets have all our agents as tools again for writing cold sales email

description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [33]:
#normal function tool

@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("sonudevmail16@gmail.com") 
    to_email = To("abhinavsarkar53@gmail.com") 
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [34]:
#some more agents as tools for subject and html conversion

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [35]:
email_tools = [subject_tool, html_tool, send_html_email]  #these three are tools for email formatting and sending

In [36]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,  #we gave all the three tools for email formatting and sending as they are rquired by the agent
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it") #this is similiar to the tool description, with this it announce to other agents it capabilities.

In [37]:
tools = [tool1, tool2, tool3]  #these are email writing agents as tools
handoffs = [emailer_agent]  #the emailer agent is the handoff agent, which will take care of formatting and sending the email

In [38]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

#### **Lets now see Structured outputs and guardrails**

In [40]:
#we will check if there is a name in the message or not, as we decided not to have any name in the mai 
class NameCheckOutput(BaseModel):
    is_name_in_message: bool   
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,  #this will ensure that the output is structured as per the NameCheckOutput class, this is used for having strucutred output 
    model="gpt-4o-mini"
)

In [42]:
#this is how we can create a guardrail function

@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [ ]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

#included name to check if the guardrail is working fine
message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

### **Now Lets Create a Project DEEP RESEARCH**

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use free and low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [46]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),  #this here make the use of the tool mandatory for the llm 
)

In [47]:
message = "Latest AI Agent frameworks in 2025"

# with trace("Search"):
#     result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

The cold sales email has been successfully sent to the CEO using the provided subject and HTML body. If you need further assistance or additional emails drafted, feel free to ask!